# Calculate network routes between origins and destinations

In [1]:
import osmnx as ox
import networkx as nx
import pandas as pd

## Load network and ODs

In [2]:
# load the origin-destination node pairs
od = pd.read_csv('data/od.csv')
len(od)

10219

In [3]:
%%time
# load the bay area's full, drivable, strongly-connected street network graph
G = ox.load_graphml('bayarea_full_simplified_strongly.graphml', folder='data/network')

Wall time: 49.7 s


## Calculate routes between origins and destinations

In [4]:
def network_distance(row):
    return nx.shortest_path_length(G, source=row['orig'], target=row['dest'], weight='length')

In [5]:
# calculate shortest path distance (meters) between rows' origin and destination nodes
od['network_distance'] = od.apply(network_distance, axis=1)

In [6]:
# save to disk
od.to_csv('data/od_distances.csv', index=False, encoding='utf-8')
od.head()

,orig,dest,network_distance
0,65335448,65303721,10456.017
1,65512687,295120796,43757.018
2,65432016,65500241,3855.411
3,2452123264,57889082,18071.854
4,65392530,65368988,22879.231
